<a href="https://colab.research.google.com/github/Eeman1113/Generating-Art-using-AI/blob/main/Generating_art_using_CC12M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check the GPU

!mkdir steps
!nvidia-smi

In [ ]:
# Install dependencies

!pip install ftfy regex requests tqdm
!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch

In [ ]:
# Download the diffusion model
# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda

!mkdir v-diffusion-pytorch/checkpoints
!curl -L https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth

In [ ]:
# Imports

import gc
import math
import sys

from IPython import display
import torch
from torchvision import utils as tv_utils
from torchvision.transforms import functional as TF
from tqdm.notebook import trange, tqdm

sys.path.append('/content/v-diffusion-pytorch')

from CLIP import clip
from diffusion import get_model, sampling, utils

In [ ]:
%pip install -q requests tqdm ftfy

%cd /content
!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch.git

%cd /content/v-diffusion-pytorch
%mkdir -p checkpoints
!curl https://v-diffusion.s3.us-west-2.amazonaws.com/cc12m_1_cfg.pth -o checkpoints/cc12m_1_cfg.pth

!./cfg_sample.py "the rise of consciousness":5 -n 4 -bs 4 --seed 0

In [ ]:
# Load the models

model = get_model('cc12m_1_cfg')()
_, side_y, side_x = model.shape
model.load_state_dict(torch.load('/content/v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth', map_location='cpu'))
model = model.half().cuda().eval().requires_grad_(False)
clip_model = clip.load(model.clip_model, jit=False, device='cpu')[0]

In [ ]:
# side_y = 512
# side_x = 512

 Only these heights and widths work: 4,8,16,32,64,128,256,512

In [ ]:
#4,8,16,32,64,128,256,512
#code to generate valid hieght and width
a=4
b=1
c=0
while b<=2000:
  c=a*2
  print(c,end=",")
  a=c
  b+=1


In [ ]:
#Only these heights and widths work: 4,8,16,32,64,128,256,512
#@title Settings
import random
prompt = 'Fantastical Landscape of the Imagination, Concept Art'  #@param {type:"string"}

height =  512#@param {type:"integer"}
width =  512#@param {type:"integer"}
side_x = width
side_y = height
#@markdown Specify the number of diffusion timesteps (default is 50, can lower for faster but lower quality sampling).
steps =   2000#@param {type:"integer"}


#@markdown Sample this many images.
n_images =   1#@param {type:"integer"}

weight = 5 

#@markdown Set to 0 for deterministic (DDIM) sampling, 1 (the default) for stochastic (DDPM) sampling, and in between to interpolate between the two. 0 is preferred for low numbers of timesteps.
eta =   0#@param {type:"number"}

#@markdown The random seed. Change this to sample different images. -1 means completely random seed!
seed =   -1#@param {type:"integer"}

#@markdown Display progress every this many timesteps.
display_every =   5#@param {type:"integer"}

save_progress_video = True #@param {type:"boolean"}

In [ ]:
#@title Run the Generator
from PIL import ImageFile, Image
import numpy as np
import os
target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()
from IPython.display import clear_output

def cfg_model_fn(x, t):
    """The CFG wrapper function."""
    n = x.shape[0]
    x_in = x.repeat([2, 1, 1, 1])
    t_in = t.repeat([2])
    clip_embed_repeat = target_embed.repeat([n, 1])
    clip_embed_in = torch.cat([torch.zeros_like(clip_embed_repeat), clip_embed_repeat])
    v_uncond, v_cond = model(x_in, t_in, clip_embed_in).chunk(2, dim=0)
    v = v_uncond + (v_cond - v_uncond) * weight
    return v
save_name = 0.00000000

def display_callback(info):
    global save_name
    save_name += 0.00000001
    nrow = math.ceil(info['pred'].shape[0]**0.5)
    grid = tv_utils.make_grid(info['pred'], nrow, padding=0)
    utils.to_pil_image(grid).save(f"/content/steps/%.8f.png" % save_name)
    
    if info['i'] % display_every == 0:
        nrow = math.ceil(info['pred'].shape[0]**0.5)
        grid = tv_utils.make_grid(info['pred'], nrow, padding=0)
        tqdm.write(f'Step {info["i"]} of {steps}:')
        display.display(utils.to_pil_image(grid))
        # utils.to_pil_image(grid).save("./steps/Test.png")
        
        tqdm.write(f'')


def run(seed):
    print("Prompt is: " + prompt)
    
    if seed == -1:
       seed = random.randint(0, 2**32)
      # print(seed)
    print("Seed is: " + str(seed))
    gc.collect()
    torch.cuda.empty_cache()
    torch.manual_seed(seed)
    x = torch.randn([n_images, 3, side_y, side_x], device='cuda')
    t = torch.linspace(1, 0, steps + 1, device='cuda')[:-1]
    step_list = utils.get_spliced_ddpm_cosine_schedule(t)
    outs = sampling.sample(cfg_model_fn, x, step_list, eta, {}, callback=display_callback)
    tqdm.write('Done!')
    for i, out in enumerate(outs):
        filename = f'out_{i}.png'
        utils.to_pil_image(out).save(filename)
        display.display(display.Image(filename))


run(seed)
frames = []
files = []
init_frame = 1 #Este es el frame donde el vídeo empezará
last_frame = steps #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

directory = '/content/steps/'
    # iterate over files in
    # that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
        # checking if it is a file
    files.append(f)
for i in range(init_frame,last_frame): #
    # save_name2 += 0.00000001
    # filename = f"steps/{save_name2}.png"
    # print(files[i])
    frames.append(Image.open(files[i]))
frames[-1].save("finalgrid.png")


if save_progress_video:

    init_frame = 1 #Este es el frame donde el vídeo empezará
    last_frame = steps #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

    min_fps = 10
    max_fps = 30

    total_frames = last_frame-init_frame

    length = 15 #Tiempo deseado del vídeo en segundos

    # import required module
    import os
    # assign directory
    directory = '/content/steps/'
    
    files = []

    # iterate over files in
    # that directory
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        files.append(f)


    files.sort()
    # print(files)


    frames = []
    save_name2 = 0.00000000
    tqdm.write('Generating video...')
    for i in range(init_frame,last_frame): #
        # save_name2 += 0.00000001
        # filename = f"steps/{save_name2}.png"
        # print(files[i])
        frames.append(Image.open(files[i]))
    # frames[-1].save("finalgrid.png")
    #fps = last_frame/10
    fps = np.clip(total_frames/length,min_fps,max_fps)

    from subprocess import Popen, PIPE
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
    for im in tqdm(frames):
        im.save(p.stdin, 'PNG')
    p.stdin.close()

    p.wait()
    print("Video finished Rendering")

In [ ]:
#@markdown Show Generated Video in Colab
from base64 import b64encode
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#end 

JS to prevent idle timeout:

Press F12 OR CTRL + SHIFT + I OR right click  -> inspect.
Then click on the console tab and paste in the following code.

```javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
```